Data cleaning and preparation by Danling.

The original dataset can cause encoding problems so I used Apple numbers to recode it.

## Data information
url: the url which fetches the data

id: the id of transaction

Lng: and Lat coordinates, using the BD09 protocol.

Cid: community id

tradeTime: the time of transaction

DOM: active days on market.Know more in https://en.wikipedia.org/wiki/Days_on_market

followers: the number of people follow the transaction.

totalPrice: the total price

price: the average price by square

square: the square of house

livingRoom: the number of living room

drawingRoom: the number of drawing room

kitchen: the number of kitchen

bathroom the number of bathroom

floor: the height of the house. 

buildingType(cat): 
1. Tower: any structure that is relatively tall in proportion to the dimensions of its base
2. Bungalow: either a single story or has a second, half, or partial story, that is built into a sloped roof
3. Combination of plate and tower
4. Plate

constructionTime: the time of construction

renovationCondition(cat): 
1. other
2. rough
3. Simplicity
4. hardcover

buildingStructure(cat): 
1. including unknown
2. mixed
3. brick and wood
4. brick and concrete
5. steel
6. steel-concrete composite

ladderRatio: the proportion between number of residents on the same floor and number of elevator of ladder. It describes how many ladders a resident have on average.

elevator(cat) have ( 1 ) or not have elevator( 0 )

fiveYearsProperty(cat): if the owner have the property for less than 5 years

Data source:
https://www.kaggle.com/datasets/ruiqurm/lianjia

In [1]:
# A couple of libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import re
import itertools



In [2]:
bj = pd.read_csv('beijing_house_recoded.csv',on_bad_lines='skip')
bj.head()

/var/folders/y7/7pfyc3l53m70d8cyk65_jhtc0000gn/T/ipykernel_67268/424483782.py:1: DtypeWarning: Columns (1,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  bj = pd.read_csv('beijing_house_recoded.csv',on_bad_lines='skip')


,url,id,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,https://bj.lianjia.com/chengjiao/101084782030....,101084782030,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,...,1.0,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,https://bj.lianjia.com/chengjiao/101086012217....,101086012217,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,...,1.0,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,https://bj.lianjia.com/chengjiao/101086041636....,101086041636,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,...,4.0,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,https://bj.lianjia.com/chengjiao/101086406841....,101086406841,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,...,1.0,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,https://bj.lianjia.com/chengjiao/101086920653....,101086920653,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,...,4.0,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0


In [3]:
# Some columns that are useless by naked eye
bj.drop(columns=['url','id'], inplace = True)

In [4]:
bj.head()

,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,131.00,2,...,1.0,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,132.38,2,...,1.0,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,198.00,3,...,4.0,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,134.00,3,...,1.0,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,81.00,2,...,4.0,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0


In [5]:
bj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318851 entries, 0 to 318850
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Lng                  318851 non-null  float64
 1   Lat                  318851 non-null  float64
 2   Cid                  318851 non-null  int64  
 3   tradeTime            318851 non-null  object 
 4   DOM                  160874 non-null  float64
 5   followers            318851 non-null  int64  
 6   totalPrice           318851 non-null  float64
 7   price                318851 non-null  int64  
 8   square               318851 non-null  float64
 9   livingRoom           318851 non-null  object 
 10  drawingRoom          318851 non-null  object 
 11  kitchen              318851 non-null  int64  
 12  bathRoom             318851 non-null  object 
 13  floor                318851 non-null  object 
 14  buildingType         316830 non-null  float64
 15  constructionTime 

In [6]:
# Dealing with unreadable characters in floor cln
def get_num(x):
    if (type(x)== int) or (type(x)== float):
        num1 = x
    else:
        listx = re.findall(r'\d+', x)
        if len(listx) > 0:
            num = listx[0]
            try: 
                num1 = int(num)
            except:
                num1 = None
        else:
            num1 = None
    return num1

In [7]:
bj['floor'] 
# the floor originally looks like this

0         ¸ß 26
1         ¸ß 22
2          ÖÐ 4
3         µ× 21
4          ÖÐ 6
          ...  
318846     ÖÐ 5
318847    ÖÐ 24
318848     ÖÐ 7
318849     ÖÐ 5
318850    ÖÐ 17
Name: floor, Length: 318851, dtype: object

In [8]:
bj['floor'] = bj['floor'].apply(get_num)

In [9]:
bj['floor']
# now it is readable!!!

0         26.0
1         22.0
2          4.0
3         21.0
4          6.0
          ... 
318846     5.0
318847    24.0
318848     7.0
318849     5.0
318850    17.0
Name: floor, Length: 318851, dtype: float64

In [10]:
bj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318851 entries, 0 to 318850
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Lng                  318851 non-null  float64
 1   Lat                  318851 non-null  float64
 2   Cid                  318851 non-null  int64  
 3   tradeTime            318851 non-null  object 
 4   DOM                  160874 non-null  float64
 5   followers            318851 non-null  int64  
 6   totalPrice           318851 non-null  float64
 7   price                318851 non-null  int64  
 8   square               318851 non-null  float64
 9   livingRoom           318851 non-null  object 
 10  drawingRoom          318851 non-null  object 
 11  kitchen              318851 non-null  int64  
 12  bathRoom             318851 non-null  object 
 13  floor                318819 non-null  float64
 14  buildingType         316830 non-null  float64
 15  constructionTime 

In [11]:
# Now clean up other object columns
print(bj['livingRoom'].unique())
print(bj['drawingRoom'].unique())
print(bj['bathRoom'].unique()) # there should never be houses with 1000+ bathrooms!!!
print(bj['constructionTime'].unique()) # there should never constructing time at 0 !!!


[2 3 1 4 5 6 9 0 7 8 '1' '4' '3' '2' '5' '7' '9' '6' '0' '#NAME?' '8']
[1 2 0 4 3 5 '0' '1' '2' '3' '4' 'ÖÐ 14' 'ÖÐ 15' 'ÖÐ 16' 'ÖÐ 6' '¸ß 14'
 '¶¥ 6' 'µÍ 6' 'µÍ 16' '¸ß 12' 'µÍ 15' '5' '¸ß 6' 'µ× 28' 'µ× 11' 'ÖÐ 24'
 'µ× 20' 'ÖÐ 22']
[1 2 3 0 4 5 6 7 2006 2003 1990 2000 1996 2005 2011 '1' '2' '0' '3' '4'
 '5' 'Î´Öª' 1994 2004 '2003' '7']
['2005' '2004' '2008' '1960' '1997' '2009' '2006' '1991' '2001' '1990'
 '2011' '2000' '1998' '2010' '1996' '1993' '2002' 'Î´Öª' '2012' '1989'
 '2003' '2007' '1994' '1984' '1992' '2014' '1985' '2013' '1999' '1979'
 '1981' '1976' '1982' '1975' '1983' '1986' '1995' '1965' '1988' '1987'
 '2015' '1955' '1980' '1978' '1958' '1970' '1956' '1977' '1964' '1963'
 '1967' '2016' '1974' '1973' '1959' '1954' '1962' '1966' '1957' '1944'
 '1972' '1971' '1953' '1968' '1961' '1950' '1952' '1933' '1969' '1' '0'
 '1906' '1934' '1914']


In [12]:
bj['livingRoom'] = bj['livingRoom'].apply(get_num)

In [13]:
print(bj['livingRoom'].unique())

[ 2.  3.  1.  4.  5.  6.  9.  0.  7.  8. nan]


In [14]:
bj['drawingRoom'] = bj['drawingRoom'].apply(get_num)
print(bj['drawingRoom'].unique())

[ 1  2  0  4  3  5 14 15 16  6 12 28 11 24 20 22]


In [15]:
bj['bathRoom'] = bj['bathRoom'].apply(get_num)
print(bj['bathRoom'].unique())

[1.000e+00 2.000e+00 3.000e+00 0.000e+00 4.000e+00 5.000e+00 6.000e+00
 7.000e+00 2.006e+03 2.003e+03 1.990e+03 2.000e+03 1.996e+03 2.005e+03
 2.011e+03       nan 1.994e+03 2.004e+03]


In [16]:
bj['constructionTime'] = bj['constructionTime'].apply(get_num)
print(bj['constructionTime'].unique())

[2.005e+03 2.004e+03 2.008e+03 1.960e+03 1.997e+03 2.009e+03 2.006e+03
 1.991e+03 2.001e+03 1.990e+03 2.011e+03 2.000e+03 1.998e+03 2.010e+03
 1.996e+03 1.993e+03 2.002e+03       nan 2.012e+03 1.989e+03 2.003e+03
 2.007e+03 1.994e+03 1.984e+03 1.992e+03 2.014e+03 1.985e+03 2.013e+03
 1.999e+03 1.979e+03 1.981e+03 1.976e+03 1.982e+03 1.975e+03 1.983e+03
 1.986e+03 1.995e+03 1.965e+03 1.988e+03 1.987e+03 2.015e+03 1.955e+03
 1.980e+03 1.978e+03 1.958e+03 1.970e+03 1.956e+03 1.977e+03 1.964e+03
 1.963e+03 1.967e+03 2.016e+03 1.974e+03 1.973e+03 1.959e+03 1.954e+03
 1.962e+03 1.966e+03 1.957e+03 1.944e+03 1.972e+03 1.971e+03 1.953e+03
 1.968e+03 1.961e+03 1.950e+03 1.952e+03 1.933e+03 1.969e+03 1.000e+00
 0.000e+00 1.906e+03 1.934e+03 1.914e+03]


So the bad data have been corrected. However, no data have been removed, no matter how seemingly unreasonable they are.

In [17]:
bj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318851 entries, 0 to 318850
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Lng                  318851 non-null  float64
 1   Lat                  318851 non-null  float64
 2   Cid                  318851 non-null  int64  
 3   tradeTime            318851 non-null  object 
 4   DOM                  160874 non-null  float64
 5   followers            318851 non-null  int64  
 6   totalPrice           318851 non-null  float64
 7   price                318851 non-null  int64  
 8   square               318851 non-null  float64
 9   livingRoom           318819 non-null  float64
 10  drawingRoom          318851 non-null  int64  
 11  kitchen              318851 non-null  int64  
 12  bathRoom             318849 non-null  float64
 13  floor                318819 non-null  float64
 14  buildingType         316830 non-null  float64
 15  constructionTime 

In [18]:
## Drop all Nan in bj_copy. 
bj_copy = bj.copy()
bj_copy.dropna(inplace = True)
bj_copy.head()

,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,131.00,2.0,...,1.0,2005.0,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,132.38,2.0,...,1.0,2004.0,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,198.00,3.0,...,4.0,2005.0,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,134.00,3.0,...,1.0,2008.0,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,81.00,2.0,...,4.0,1960.0,2,2,0.333,0.0,1.0,1.0,1,62588.0


In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    bj_copy.drop(labels=['totalPrice','tradeTime'], axis=1),
    bj_copy['totalPrice'],
    test_size=0.2,
    random_state=23)

X_train.shape, X_test.shape

((121812, 22), (30454, 22))

In [20]:
def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = linear_model.LinearRegression()
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y,model_k.predict(X)) * len(Y)
    R_squared = model_k.score(X,Y)
    return RSS, R_squared

In [21]:
k = 22

remaining_features = list(X_train.columns.values)
features = []
RSS_list, R_squared_list = [np.inf], [np.inf] #Due to 1 indexing of the loop...
features_list = dict()

for i in range(1,k+1):
    best_RSS = np.inf
    
    for combo in itertools.combinations(remaining_features,1):

            RSS = fit_linear_reg(X_train[list(combo) + features],y_train)   #Store temp result 

            if RSS[0] < best_RSS:
                best_RSS = RSS[0]
                best_R_squared = RSS[1] 
                best_feature = combo[0]

    #Updating variables for next loop
    features.append(best_feature)
    remaining_features.remove(best_feature)
    
    #Saving values for plotting
    RSS_list.append(best_RSS)
    R_squared_list.append(best_R_squared)
    features_list[i] = features.copy()
    

In [22]:
df1 = pd.concat([pd.DataFrame({'features':features_list}),pd.DataFrame({'RSS':RSS_list, 'R_squared': R_squared_list})], axis=1, join='inner')
df1['numb_features'] = df1.index

In [23]:
#Initializing useful variables
m = len(y_train)
p = 11
hat_sigma_squared = (1/(m - p -1)) * min(df1['RSS'])

#Computing
df1['C_p'] = (1/m) * (df1['RSS'] + 2 * df1['numb_features'] * hat_sigma_squared )
df1['AIC'] = (1/(m*hat_sigma_squared)) * (df1['RSS'] + 2 * df1['numb_features'] * hat_sigma_squared )
df1['BIC'] = (1/(m*hat_sigma_squared)) * (df1['RSS'] +  np.log(m) * df1['numb_features'] * hat_sigma_squared )
df1['R_squared_adj'] = 1 - ( (1 - df1['R_squared'])*(m-1)/(m-df1['numb_features'] -1))
df1

,features,RSS,R_squared,numb_features,C_p,AIC,BIC,R_squared_adj
1,[price],4.955133e+09,0.370273,1,40678.659710,5.134655,5.134735,0.370268
2,"[price, square]",1.004046e+09,0.872400,2,8242.845355,1.040451,1.040611,0.872398
3,"[price, square, buildingStructure]",9.829698e+08,0.875079,3,8069.954715,1.018628,1.018867,0.875075
4,"[price, square, buildingStructure, communityAv...",9.801856e+08,0.875432,4,8047.227933,1.015760,1.016079,0.875428
5,"[price, square, buildingStructure, communityAv...",9.768630e+08,0.875855,5,8020.081637,1.012333,1.012732,0.875850
6,"[price, square, buildingStructure, communityAv...",9.744010e+08,0.876168,6,8000.000621,1.009798,1.010277,0.876161
7,"[price, square, buildingStructure, communityAv...",9.725402e+08,0.876404,7,7984.854769,1.007887,1.008445,0.876397
8,"[price, square, buildingStructure, communityAv...",9.706819e+08,0.876640,8,7969.728979,1.005977,1.006615,0.876632
9,"[price, square, buildingStructure, communityAv...",9.692372e+08,0.876824,9,7957.999443,1.004497,1.005214,0.876815
10,"[price, square, buildingStructure, communityAv...",9.678355e+08,0.877002,10,7946.622164,1.003061,1.003858,0.876992


In [36]:
# make dummies for categorical variables
dum_buildingType = pd.get_dummies(bj_copy.buildingType, prefix='building_type')
dum_renovationCondition = pd.get_dummies(bj_copy.renovationCondition, prefix='renovationCondition')
dum_buildingStructure = pd.get_dummies(bj_copy.buildingStructure, prefix='buildingStructure')

bj_copy = pd.concat([bj_copy,dum_buildingStructure, dum_buildingType, dum_renovationCondition], axis = 1)
bj_copy

,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,...,buildingStructure_5,buildingStructure_6,building_type_1.0,building_type_2.0,building_type_3.0,building_type_4.0,renovationCondition_1,renovationCondition_2,renovationCondition_3,renovationCondition_4
0,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,131.00,2.0,...,0,1,1,0,0,0,0,0,1,0
1,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,132.38,2.0,...,0,1,1,0,0,0,0,0,0,1
2,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,198.00,3.0,...,0,1,0,0,0,1,0,0,1,0
3,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,134.00,3.0,...,0,1,1,0,0,0,1,0,0,0
4,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,81.00,2.0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318842,116.508181,39.807662,1111027374693,2016-06-10,1.0,3,124.0,37702,32.89,1.0,...,0,1,1,0,0,0,1,0,0,0
318844,116.550348,39.755625,1111027381517,2016-06-10,1.0,6,268.0,22709,118.02,2.0,...,0,1,0,0,0,1,0,0,0,1
318845,116.546899,39.755236,1111027377053,2016-06-09,1.0,8,355.5,26491,134.20,3.0,...,0,1,0,0,1,0,0,0,0,1
318846,116.546899,39.755236,1111027377053,2016-06-10,1.0,8,350.0,20580,170.07,3.0,...,0,1,0,0,0,1,1,0,0,0
